## Problem 1

In Matlab or Python, implement a version of the TSQR that divides an input matrix up into four blocks of rows (using row sub-indexing) and computes the QR-factorisation in the way shown in the lectures on communication-avoiding factorisations.

TSQR defines a family of algorithms, in which the QR factorization of A is obtained by performing a sequence of QR factorizations until the lower trapezoidal part of A is annihilated and the final R factor is obtained. The QR factorizations are performed on block rows of A and on previously obtained R factors, stacked atop one another. We call the pattern followed during this sequence of QR factorizations a reduction tree.

### Sequential TSQR

The first set of algorithms, “Tall Skinny QR” (TSQR), are for matrices for which the number of rows is much larger than the number of columns, and which have their rows dis-
tributed over processors in a one-dimensional (1-D) block row layout.

Sequential TSQR uses a similar factorization process, but with a “flat tree” (a linear chain). We start with the same block row decomposition as with parallel TSQR,
but begin with a QR factorization of $A_0$, rather than of all the block rows:

$$
A =
\begin{pmatrix}
A_0 \\
A_1 \\
A_2 \\
A_3
\end{pmatrix}
=
\begin{pmatrix}
Q_{00} R_{00} \\
A_1 \\
A_2 \\
A_3
\end{pmatrix}
$$

This is “stage 0” of the computation, hence the second subscript 0 of the $Q$ and $R$ factor. We then combine $R_{00}$ and $A_1$ using a QR factorization:

$$
\begin{pmatrix}
R_{00} \\
A_1 \\
A_2 \\
A_3
\end{pmatrix}
=
\begin{pmatrix}
R_{00} \\
A_1 \\
\hline
A_2 \\
A_3
\end{pmatrix}
=
\begin{pmatrix}
Q_{01} R_{01} \\
\hline
A_2 \\
A_3
\end{pmatrix}
$$

We continue this process until we run out of $A_i$ factors. Here, the $A_i$ blocks are $m/P \times n$. If we were to compute all the above $Q$ factors explicitly as square matrices, which we do not, then $Q_{00}$ would be $m/P \times m/P$ and $Q_{0j}$ for $j > 0$ would be $2m/P \times 2m/P$ . The final R factor, as in the parallel case, would be $m \times n$ upper triangular (or $n \times n$ upper triangular in a “thin QR”).

In [13]:
import numpy as np


# ============================================================
# Sequential implementation of TSQR
# ============================================================
p = 4                                       # 4 block rows
m = 4 * p                                   # So each block has 4 rows
n = 3                                       # So each block has 3 columns (m > n)
scaling_factor = 100                        # Scales elements, otherwise [0, 1)
np.random.seed(42)                          # Seeds random number generator
A = scaling_factor * np.random.rand(m, n)   # Matrix
print(f"A ({m} x {n}):\n{A}\n")

# Partioning A into p blocks
mb = A.shape[0] // p
A_partitioned = [None] * p
for i in range(p):
  A_partitioned[i] = A[mb * i: mb * (i + 1), :]
  print(f"A_{i} ({mb} x {n}):")
  print(f"{A_partitioned[i]}\n")

# Sequential TSQR with FULL QR (mode='complete')
# Full QR gives square orthogonal Q matrices, which we can embed into m x m
Qs_full   = [None] * p
R_current = None

for i in range(p):
    if i == 0:
        Q_f, R_f   = np.linalg.qr(A_blocks[i], mode='complete')
        Qs_full[i] = Q_f                                        # mb x mb  (square)
        R_current  = R_f[:n, :]                                 # n x n (thin R, upper triangular)
        print(f"R_0{i} ({R_current.shape[0]} x {R_current.shape[1]}):\n{R_current}\n")
    else:
        stacked    = np.vstack((R_current, A_blocks[i]))        # (n + mb) x n
        Q_f, R_f   = np.linalg.qr(stacked, mode='complete')
        Qs_full[i] = Q_f                                        # (n + mb) x (n + mb)  (square)
        R_current  = R_f[:n, :]                                 # n x n
        print(f"R_0{i} ({R_current.shape[0]} x {R_current.shape[1]}):\n{R_current}\n")

A (16 x 3):
[[37.45401188 95.07143064 73.19939418]
 [59.86584842 15.60186404 15.59945203]
 [ 5.80836122 86.61761458 60.11150117]
 [70.80725778  2.05844943 96.99098522]
 [83.24426408 21.23391107 18.18249672]
 [18.34045099 30.4242243  52.47564316]
 [43.19450186 29.12291402 61.18528947]
 [13.94938607 29.21446485 36.63618433]
 [45.60699842 78.51759614 19.96737822]
 [51.42344384 59.24145689  4.64504127]
 [60.75448519 17.05241237  6.5051593 ]
 [94.88855373 96.56320331 80.83973481]
 [30.46137692  9.7672114  68.42330265]
 [44.01524937 12.20382348 49.51769101]
 [ 3.43885211 90.93204021 25.87799816]
 [66.25222844 31.17110761 52.00680212]]

A_0 (4 x 3):
[[37.45401188 95.07143064 73.19939418]
 [59.86584842 15.60186404 15.59945203]
 [ 5.80836122 86.61761458 60.11150117]
 [70.80725778  2.05844943 96.99098522]]

A_1 (4 x 3):
[[83.24426408 21.23391107 18.18249672]
 [18.34045099 30.4242243  52.47564316]
 [43.19450186 29.12291402 61.18528947]
 [13.94938607 29.21446485 36.63618433]]

A_2 (4 x 3):
[[45.60

In [10]:
# ============================================================
# Reconstructing A
# ============================================================
# Build embedded m x m orthogonal matrices
# Each M_i embeds Q_{0i} into an m x m identity matrix.
def embed_Q(Q, indices, m):
    """Embed square matrix Q into m x m identity at the given row/col indices."""
    M = np.eye(m)
    for a, row_a in enumerate(indices):
        for b, row_b in enumerate(indices):
            M[row_a, row_b] = Q[a, b]
    return M

Ms = []
for i in range(p):
    if i == 0:
        indices = list(range(mb))
    else:
        indices = list(range(n)) + list(range(i * mb, (i + 1) * mb))
    print(f"M{i+1} embeds Q_0{i} ({Qs_full[i].shape}) at rows {indices[:3]}...{indices[-3:]}")
    Ms.append(embed_Q(Qs_full[i], indices, m))

# Embed R_final into m x n  (zeros below)
R_embedded = np.zeros((m, n))
R_embedded[:n, :] = R_final

# Reconstruct A = M1 @ M2 @ M3 @ M4 @ R_embedded
computed_A = Ms[0] @ Ms[1] @ Ms[2] @ Ms[3] @ R_embedded

print(f"\nMax reconstruction error: {np.max(np.abs(computed_A - A)):.4e}\n")
print(f"A:\n{np.round(A, 4)}\n")
print(f"computed_A:\n{np.round(computed_A, 4)}\n")

M1 embeds Q_00 ((4, 4)) at rows [0, 1, 2]...[1, 2, 3]
M2 embeds Q_01 ((7, 7)) at rows [0, 1, 2]...[5, 6, 7]
M3 embeds Q_02 ((7, 7)) at rows [0, 1, 2]...[9, 10, 11]
M4 embeds Q_03 ((7, 7)) at rows [0, 1, 2]...[13, 14, 15]

Max reconstruction error: 4.2633e-14

A:
[[37.454  95.0714 73.1994]
 [59.8658 15.6019 15.5995]
 [ 5.8084 86.6176 60.1115]
 [70.8073  2.0584 96.991 ]
 [83.2443 21.2339 18.1825]
 [18.3405 30.4242 52.4756]
 [43.1945 29.1229 61.1853]
 [13.9494 29.2145 36.6362]
 [45.607  78.5176 19.9674]
 [51.4234 59.2415  4.645 ]
 [60.7545 17.0524  6.5052]
 [94.8886 96.5632 80.8397]
 [30.4614  9.7672 68.4233]
 [44.0152 12.2038 49.5177]
 [ 3.4389 90.932  25.878 ]
 [66.2522 31.1711 52.0068]]

computed_A:
[[37.454  95.0714 73.1994]
 [59.8658 15.6019 15.5995]
 [ 5.8084 86.6176 60.1115]
 [70.8073  2.0584 96.991 ]
 [83.2443 21.2339 18.1825]
 [18.3405 30.4242 52.4756]
 [43.1945 29.1229 61.1853]
 [13.9494 29.2145 36.6362]
 [45.607  78.5176 19.9674]
 [51.4234 59.2415  4.645 ]
 [60.7545 17.0524  6.

In [14]:
# Sequential TSQR with Thin QR (mode='reduced')
Qs_thin   = [None] * p
R_current = None

for i in range(p):
    if i == 0:
        Qs_thin[i], R_current = np.linalg.qr(A_blocks[i], mode='reduced')
        print(f"R_0{i} ({R_current.shape[0]} x {R_current.shape[1]}):\n{R_current}\n")
    else:
        stacked = np.vstack((R_current, A_blocks[i]))
        Qs_thin[i], R_current = np.linalg.qr(stacked, mode='reduced')
        print(f"R_0{i} ({R_current.shape[0]} x {R_current.shape[1]}):\n{R_current}\n")

R_final = R_current  # n × n

# Reconstructing A
# Unroll the chain (no embedding needed)
R2 = Qs_thin[3][:n, :] @ R_final
R1 = Qs_thin[2][:n, :] @ R2
R0 = Qs_thin[1][:n, :] @ R1

A0_rec = Qs_thin[0]        @ R0
A1_rec = Qs_thin[1][n:, :] @ R1
A2_rec = Qs_thin[2][n:, :] @ R2
A3_rec = Qs_thin[3][n:, :] @ R_final

computed_A = np.vstack([A0_rec, A1_rec, A2_rec, A3_rec])
print(f"Max reconstruction error: {np.max(np.abs(computed_A - A)):.2e}")

R_00 (3 x 3):
[[-100.1704928   -51.34930187 -108.73761435]
 [   0.          118.96256828   59.05485274]
 [   0.            0.           57.53949594]]

R_01 (3 x 3):
[[ 139.14186155   65.65065092  118.74382109]
 [   0.         -124.72761109  -77.35356272]
 [   0.            0.          -81.90613682]]

R_02 (3 x 3):
[[-191.78052981 -135.36754049 -134.2041117 ]
 [   0.          144.33438272   62.61083117]
 [   0.            0.          108.89938755]]

R_03 (3 x 3):
[[ 209.87184841  139.00574603  159.79320034]
 [   0.         -171.22731452  -59.79456032]
 [   0.            0.         -123.24288373]]

Max reconstruction error: 5.68e-14


References:
- [1] Jammes Demmel, Laura Grigori, Mark Hoemmen, and Julien Langou. Implementing Communication-Optimal Parallel And Sequential QR Factorizations. https://arxiv.abs/pdf/0809.2407, 2008. arXiv:0809.2407 [math.NA]